In [1]:
# Cell 0 — Imports & paths (RAW → new_results)
# NOTE: Configure paths once. Old data in data/raw/ stays untouched; all new outputs go to new_results/.

from pathlib import Path
import zipfile, shutil
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_integer_dtype

# Repo root (your project folder)
ROOT = Path("/home/adetayo/Documents/CSCI Forms/Adetayo Research/Cancer Screening Behavior/new_results/publication")

# 🔹 Inputs live in the repo ROOT (per your note)
fresh_zip_path       = ROOT / "data_set.zip"  # <-- ZIP is here
rename_template_path = ROOT / "nlsy_variable_names_template_fresh.csv"  # adjust if yours is elsewhere

# (Optional) fallback if the template isn't in ROOT
if not rename_template_path.exists():
    alt = ROOT / "data" / "raw" / "nlsy_variable_names_template_fresh.csv"
    if alt.exists():
        rename_template_path = alt

# 🔹 Outputs go to a fresh bucket (keeps old stuff untouched)
RUN_ROOT   = ROOT / "new_results"
INTERIM    = RUN_ROOT / "interim"
PROCESSED  = RUN_ROOT / "processed"
RESULTS    = RUN_ROOT / "results" / "tables"

for p in [INTERIM, PROCESSED, RESULTS]:
    p.mkdir(parents=True, exist_ok=True)

# Output file paths (all inside new_results/)
fresh_extract_path    = INTERIM / "Data_Reloaded"
renamed_csv_path      = INTERIM / "nlsy_data_with_renamed_columns.csv"
females_only_path     = INTERIM / "nlsy_data_females_only.csv"
negative_summary_path = RESULTS / "remaining_negative_summary.txt"
final_features_path   = PROCESSED / "final_dataset.csv"

# Keep notebook output tidy
pd.options.mode.chained_assignment = None



In [2]:
# Cell 1 — Helper functions
# NOTE: Define once to keep later cells short; easy to move into src/ later.

def extract_zip(zip_path: Path, out_dir: Path, clean=True):
    """Extract the input ZIP to out_dir. If clean=True, wipes the folder first."""
    if clean and out_dir.exists():
        shutil.rmtree(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(out_dir)

def load_and_rename(csv_path: Path, rename_csv: Path) -> pd.DataFrame:
    """Load CSV and apply rename mapping from template (variable_name -> new_name)."""
    df = pd.read_csv(csv_path)
    rn = pd.read_csv(rename_csv)
    rn["variable_name"] = rn["variable_name"].astype(str).str.strip()
    rn["new_name"] = rn["new_name"].fillna("").astype(str).str.strip()
    rename_dict = dict(
        rn.loc[rn["new_name"] != "", ["variable_name","new_name"]]
          .set_index("variable_name")["new_name"]
    )
    return df.rename(columns=rename_dict)

def mask_negatives(df: pd.DataFrame, cols):
    """Replace negatives with NaN in the provided columns."""
    out = df.copy()
    if cols:
        out.loc[:, cols] = out[cols].mask(out[cols] < 0)
    return out

def invalidate_codes(df: pd.DataFrame, cols=("educ","mother_educ"), invalid=(0,93,94,95)):
    """Set invalid codes (e.g., 0, 93–95) to NaN for education fields."""
    out = df.copy()
    for c in cols:
        if c in out.columns:
            out[c] = out[c].mask(out[c].isin(invalid))
    return out

def wide_to_long_years(df: pd.DataFrame, id_col="id_code", years=range(2002,2023)):
    """Convert *_YYYY columns to long format with a 'year' column."""
    df = df.copy()
    long_columns = [c for c in df.columns if any(str(y) in c for y in years)]
    stubs = sorted({c.rsplit("_",1)[0] for c in long_columns if c.rsplit("_",1)[-1].isdigit()})
    dfl = pd.wide_to_long(df, stubnames=stubs, i=id_col, j="year", sep="_", suffix="\\d+").reset_index()
    return dfl

def add_time_vars(df: pd.DataFrame, allowed_years=(2008,2010,2012,2014,2016,2018), base_year=2008):
    """Create time_step (0..K) and time_elapsed; filter to allowed years."""
    out = df.copy()
    mapping = {y:i for i,y in enumerate(sorted(allowed_years))}
    out["time_step"] = out["year"].map(mapping)
    out["time_elapsed"] = out["year"] - base_year
    ymin, ymax = min(allowed_years), max(allowed_years)
    out = out.loc[out["year"].between(ymin, ymax)].sort_values(["id_code","year"])
    return out

def add_income_lags(df, inc_col="inc", group="id_code", time_col="year"):
    out = df.sort_values([group, time_col]).copy()

    s = pd.to_numeric(out[inc_col], errors="coerce").astype(float)

    # log(1+y) defined for y >= 0; negatives -> NaN
    out["income_log"] = np.where(s >= 0, np.log1p(s), np.nan)

    out["income_log_lag1"] = out.groupby(group)["income_log"].shift(1)
    out["income_log_lag2"] = out.groupby(group)["income_log"].shift(2)
    return out


def add_pap_lags(df: pd.DataFrame, col="pap_smear", group="id_code", max_lag=3):
    """Create pap_smear_lag1..lagK by person."""
    out = df.copy()
    if col in out.columns:
        for k in range(1, max_lag+1):
            out[f"{col}_lag{k}"] = out.groupby(group)[col].shift(k)
    return out

def add_interactions(df: pd.DataFrame):
    """Build interactions; robust to self_assessment vs self_assement typos."""
    out = df.copy()
    s50 = "self_assessment50" if "self_assessment50" in out.columns else ("self_assement50" if "self_assement50" in out.columns else None)
    s60 = "self_assessment60" if "self_assessment60" in out.columns else ("self_assement60" if "self_assement60" in out.columns else None)

    if "income_log" in out.columns:
        if s50 in out.columns:
            out["income_log_self50"] = out["income_log"] * out[s50]
            out["income_log_lag2_self50"] = out["income_log_lag2"] * out[s50]
        if s60 in out.columns:
            out["income_log_self60"] = out["income_log"] * out[s60]
            out["income_log_lag2_self60"] = out["income_log_lag2"] * out[s60]
        for col in ["region","health_provider","pap_smear_lag1","health_plan"]:
            if col in out.columns:
                out[f"income_log_{col}"] = out["income_log"] * out[col]

    if {"region","year"}.issubset(out.columns):
        out["region_year"] = out["region"] * out["year"]
    if {"region","health_plan"}.issubset(out.columns):
        out["region_health_plan"] = out["region"] * out["health_plan"]
    if {"race","health_plan"}.issubset(out.columns):
        out["race_health_plan"] = out["race"] * out["health_plan"]
    if {"educ","mother_educ"}.issubset(out.columns):
        out["educ_mother_educ"] = out["educ"] * out["mother_educ"]
    if {"income_log", "pap_smear_lag1"}.issubset(out.columns):
        out['income_log_pap_smear_lag1'] = out['income_log'] * out['pap_smear_lag1']
    return out


In [3]:
# Cell 2 — Extract from RAW → find & load CSV into DataFrame
# NOTE: Extracts into new_results/interim/Data_Reloaded/. Prefers Other_Demo.csv, otherwise selects the only CSV found.

# Sanity check before extraction
assert fresh_zip_path.exists(), f"ZIP not found: {fresh_zip_path}"

# Extract (safe to clean; it's a NEW folder)
extract_zip(fresh_zip_path, fresh_extract_path, clean=True)

# Locate CSV
candidates = list(fresh_extract_path.rglob("*.csv"))
target_csv = next((c for c in candidates if c.name.lower() == "other_demo.csv"), None)
if target_csv is None and len(candidates) == 1:
    target_csv = candidates[0]
if target_csv is None:
    raise FileNotFoundError(
        f"Could not find 'Other_Demo.csv' after extraction.\n"
        f"Found {len(candidates)} CSVs: {[c.name for c in candidates]}"
    )

# Load raw
df_raw = pd.read_csv(target_csv)
df_raw.head()


,H0015900,H0052200,R0000100,R0000500,R0006500,R0173600,R0214700,R0214800,R7598800,R7703700,...,T9180600,T9180700,T9184600,T9299700,T9300100,T9300200,T9300400,T9302300,T9307900,T9900000
0,-4,-4,1,58,8,5,3,2,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,12
1,3,3,2,59,5,5,3,2,1,50450,...,1,1,1,46260,1,2,63,2,2,12
2,2,3,3,61,10,5,3,2,1,-2,...,1,1,1,81000,4,2,61,3,0,12
3,4,4,4,62,11,5,3,2,-5,-5,...,1,1,1,108895,4,2,60,3,0,14
4,-4,-4,5,59,12,1,3,1,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,18


In [4]:
# Cell 3 — Rename columns (template in RAW) → save to new_results/interim

df_renamed = load_and_rename(target_csv, rename_template_path)
df_renamed.to_csv(renamed_csv_path, index=False)
print(f"✅ Renamed file saved to: {renamed_csv_path}")
df_renamed.head()


✅ Renamed file saved to: /home/adetayo/Documents/CSCI Forms/Adetayo Research/Cancer Screening Behavior/new_results/publication/new_results/interim/nlsy_data_with_renamed_columns.csv


,self_assement50,self_assement60,id_code,birth_year,mother_educ,sample_id,race,gender,last_exam_2002,inc_2002,...,health_provider_2022,last_exam_2022,health_plan_2022,inc_2022,region_2022,marital_status_2022,age_2022,smsa_2022,hh_children_2022,educ
0,-4,-4,1,58,8,5,3,2,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,12
1,3,3,2,59,5,5,3,2,1,50450,...,1,1,1,46260,1,2,63,2,2,12
2,2,3,3,61,10,5,3,2,1,-2,...,1,1,1,81000,4,2,61,3,0,12
3,4,4,4,62,11,5,3,2,-5,-5,...,1,1,1,108895,4,2,60,3,0,14
4,-4,-4,5,59,12,1,3,1,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,18


In [5]:
# Cell 4 — Filter females

df_females = df_renamed.loc[df_renamed["gender"] == 2].copy()
print("Shape after gender==2 filter:", df_females.shape)
df_females.head()

Shape after gender==2 filter: (6283, 117)


,self_assement50,self_assement60,id_code,birth_year,mother_educ,sample_id,race,gender,last_exam_2002,inc_2002,...,health_provider_2022,last_exam_2022,health_plan_2022,inc_2022,region_2022,marital_status_2022,age_2022,smsa_2022,hh_children_2022,educ
0,-4,-4,1,58,8,5,3,2,-5,-5,...,-5,-5,-5,-5,-5,-5,-5,-5,-5,12
1,3,3,2,59,5,5,3,2,1,50450,...,1,1,1,46260,1,2,63,2,2,12
2,2,3,3,61,10,5,3,2,1,-2,...,1,1,1,81000,4,2,61,3,0,12
3,4,4,4,62,11,5,3,2,-5,-5,...,1,1,1,108895,4,2,60,3,0,14
7,2,4,8,58,9,6,3,2,1,36700,...,1,1,1,55056,1,2,64,2,0,14


In [6]:
# Cell 5 — Drop rows with negatives in screening cols
# NOTE: Any negative in pap_smear_* or mammogram_* drops the row (as in your original script).

screening_cols = [c for c in df_females.columns if ("pap_smear_" in c or "mammogram_" in c)]
if screening_cols:
    df_females = df_females.loc[~df_females[screening_cols].lt(0).any(axis=1)].copy()
print("Kept rows after screening negative drop:", df_females.shape[0])


Kept rows after screening negative drop: 3087


In [7]:
# Cell 6 — Mask negatives in self-assessment cols (robust, no FutureWarning)
# NOTE: Cast each target column to float first, then mask negatives -> NaN.


self_cols = [
    c for c in df_females.columns
    if ("self_assessment50" in c or "self_assessment60" in c
        or "self_assement50" in c or "self_assement60" in c)
]

# keep only numeric self cols
num_self_cols = [c for c in self_cols if is_numeric_dtype(df_females[c])]

changed = []
for c in num_self_cols:
    s = pd.to_numeric(df_females[c], errors="coerce").astype("float64")
    s = s.where(s >= 0, np.nan)
    df_females[c] = s
    changed.append(c)

print("Self-assessment cleaned (per-column float cast):", changed if changed else "none")
df_females.head()


Self-assessment cleaned (per-column float cast): ['self_assement50', 'self_assement60']


,self_assement50,self_assement60,id_code,birth_year,mother_educ,sample_id,race,gender,last_exam_2002,inc_2002,...,health_provider_2022,last_exam_2022,health_plan_2022,inc_2022,region_2022,marital_status_2022,age_2022,smsa_2022,hh_children_2022,educ
1,3.0,3.0,2,59,5,5,3,2,1,50450,...,1,1,1,46260,1,2,63,2,2,12
7,2.0,4.0,8,58,9,6,3,2,1,36700,...,1,1,1,55056,1,2,64,2,0,14
15,3.0,2.0,16,58,12,5,3,2,2,86400,...,1,1,1,107000,1,3,64,2,1,13
20,1.0,1.0,21,61,12,5,3,2,4,175460,...,1,1,1,134000,1,2,61,2,0,16
24,2.0,2.0,25,59,14,5,3,2,1,390662,...,1,1,1,225000,1,2,62,2,0,14


In [8]:
# Cell 7 — Mask negatives in other numeric predictors (dtype-safe, no warnings)
# NOTE: For all other numeric columns (excluding screening/self/gender), set negatives to missing.
#       - int columns -> cast to pandas nullable Int64 and use pd.NA
#       - float columns -> keep float and use NaN

exclude_cols = set((screening_cols or []) + (self_cols or []) + ["gender"])
num_cols = [c for c in df_females.columns if is_numeric_dtype(df_females[c])]
other_num = [c for c in num_cols if c not in exclude_cols]

changed_int, changed_float = [], []
for c in other_num:
    s = df_females[c]
    if is_integer_dtype(s):
        # Cast to nullable Int64 first, then mask with pd.NA
        s2 = s.astype("Int64").mask(s < 0, pd.NA)
        df_females[c] = s2
        changed_int.append(c)
    else:
        # Ensure float dtype, then mask with NaN
        s2 = pd.to_numeric(s, errors="coerce").astype("float64")
        s2 = s2.where(s2 >= 0, np.nan)
        df_females[c] = s2
        changed_float.append(c)

print("Other numeric columns cleaned:",
      {"int->Int64": len(changed_int), "float": len(changed_float), "total": len(other_num)})



Other numeric columns cleaned: {'int->Int64': 102, 'float': 0, 'total': 102}


In [9]:
# Cell 8 — Invalidate education/mother_educ codes

# NOTE: Converts invalid codes (0, 93–95) to NaN.

df_females = invalidate_codes(df_females, cols=("educ","mother_educ"), invalid=(0,93,94,95))
df_females[["educ","mother_educ"]].head()

,educ,mother_educ
1,12,5
7,14,9
15,13,12
20,16,12
24,14,14


In [10]:
# Cell 9 — Save cleaned females (new_results/interim)

# NOTE: Persist this intermediate artifact for downstream steps.

df_females.to_csv(females_only_path, index=False)
print(f"✅ Final cleaned female-only dataset saved to:\n{females_only_path}")

✅ Final cleaned female-only dataset saved to:
/home/adetayo/Documents/CSCI Forms/Adetayo Research/Cancer Screening Behavior/new_results/publication/new_results/interim/nlsy_data_females_only.csv


In [11]:
# Cell 10 — Negative summary (new_results/results/tables)
# NOTE: Quick sanity check that no negatives remain; writes a small TSV.

neg_counts = (df_females.select_dtypes(include=[np.number]) < 0).sum()
summary_df = neg_counts.reset_index()
summary_df.columns = ["Variable", "Count_Negative_Values"]
summary_df.sort_values("Count_Negative_Values", ascending=False, inplace=True)
summary_df.to_csv(negative_summary_path, index=False, sep="\t")
print(f"📊 Negative value summary saved to: {negative_summary_path}")
summary_df.head(10)


📊 Negative value summary saved to: /home/adetayo/Documents/CSCI Forms/Adetayo Research/Cancer Screening Behavior/new_results/publication/new_results/results/tables/remaining_negative_summary.txt


,Variable,Count_Negative_Values
0,self_assement50,0
74,health_provider_2016,0
86,health_provider_2018,0
85,hh_children_2016,0
84,smsa_2016,0
83,age_2016,0
82,marital_status_2016,0
81,region_2016,0
80,inc_2016,0
79,health_plan_2016,0


In [12]:
# Cell 11 — Wide → long by year
# NOTE: Converts *_YYYY columns into a tidy long format with a year column.

years = range(2002, 2023)
df_long = wide_to_long_years(df_females, id_col="id_code", years=years)
df_long.head()

,id_code,year,race,mother_educ,self_assement50,sample_id,educ,birth_year,gender,self_assement60,...,health_plan,health_provider,hh_children,inc,last_exam,mammogram,marital_status,pap_smear,region,smsa
0,2,2004,3,5,3.0,5,12,59,2,3.0,...,1,<NA>,2,55000,1,NaN,2,NaN,1,2
1,8,2004,3,9,2.0,6,14,58,2,4.0,...,1,<NA>,1,39200,1,NaN,3,NaN,1,3
2,16,2004,3,12,3.0,5,13,58,2,2.0,...,1,<NA>,3,134636,2,NaN,2,NaN,1,2
3,21,2004,3,12,1.0,5,16,61,2,1.0,...,1,<NA>,0,170025,3,NaN,2,NaN,1,2
4,25,2004,3,14,2.0,5,14,59,2,2.0,...,1,<NA>,2,408473,1,NaN,2,NaN,1,2


In [13]:
# Cell 12 — Add time variables & restrict to 2008–2018
# NOTE: Builds time_step and time_elapsed, then keeps only your target waves.

df_timed = add_time_vars(df_long, allowed_years=(2008,2010,2012,2014,2016,2018), base_year=2008)
df_timed[["id_code","year","time_step","time_elapsed"]].head()

,id_code,year,time_step,time_elapsed
6174,2,2008,0.0,0
9261,2,2010,1.0,2
12348,2,2012,2.0,4
15435,2,2014,3.0,6
18522,2,2016,4.0,8


In [14]:
# Cell 13 — Income log and lags
# NOTE: Creates income_log and its 1–2 period lags per individual.

df_lagged = add_income_lags(df_timed, inc_col="inc", group="id_code")
df_lagged[["id_code","year","inc","income_log","income_log_lag1","income_log_lag2"]].head()

,id_code,year,inc,income_log,income_log_lag1,income_log_lag2
6174,2,2008,7000,8.853808,NaN,NaN
9261,2,2010,16800,9.729194,8.853808,NaN
12348,2,2012,43000,10.668979,9.729194,8.853808
15435,2,2014,35400,10.474495,10.668979,9.729194
18522,2,2016,<NA>,NaN,10.474495,10.668979


In [15]:
# Cell 14 — Pap smear lags
# NOTE: Constructs pap_smear_lag1..lag3 per individual.

df_lagged = add_pap_lags(df_lagged, col="pap_smear", group="id_code", max_lag=3)
df_lagged[["id_code","year","pap_smear","pap_smear_lag1","pap_smear_lag2","pap_smear_lag3"]].head()


,id_code,year,pap_smear,pap_smear_lag1,pap_smear_lag2,pap_smear_lag3
6174,2,2008,1.0,NaN,NaN,NaN
9261,2,2010,1.0,1.0,NaN,NaN
12348,2,2012,1.0,1.0,1.0,NaN
15435,2,2014,0.0,1.0,1.0,1.0
18522,2,2016,1.0,0.0,1.0,1.0


In [16]:
# Cell 15 — Feature interactions

# NOTE: Builds interaction terms (income × self assessments, region, etc.); typo-tolerant.

df_feat = add_interactions(df_lagged)
df_feat.head()

,id_code,year,race,mother_educ,self_assement50,sample_id,educ,birth_year,gender,self_assement60,...,income_log_self60,income_log_lag2_self60,income_log_region,income_log_health_provider,income_log_pap_smear_lag1,income_log_health_plan,region_year,region_health_plan,race_health_plan,educ_mother_educ
6174,2,2008,3,5,3.0,5,12,59,2,3.0,...,26.561425,NaN,8.853808,8.853808,NaN,8.853808,2008,1,3,60
9261,2,2010,3,5,3.0,5,12,59,2,3.0,...,29.187581,NaN,9.729194,9.729194,9.729194,9.729194,2010,1,3,60
12348,2,2012,3,5,3.0,5,12,59,2,3.0,...,32.006936,26.561425,10.668979,10.668979,10.668979,10.668979,2012,1,3,60
15435,2,2014,3,5,3.0,5,12,59,2,3.0,...,31.423486,29.187581,10.474495,10.474495,10.474495,10.474495,2014,1,3,60
18522,2,2016,3,5,3.0,5,12,59,2,3.0,...,NaN,32.006936,<NA>,<NA>,NaN,<NA>,2016,1,3,60


In [17]:
# Cell 16 — Save final features (new_results/processed)
# NOTE: Modeling-ready dataset saved alongside your other new artifacts.

df_feat.to_csv(final_features_path, index=False)
print("✅ Final dataset with lags and interactions saved to:\n", final_features_path)

✅ Final dataset with lags and interactions saved to:
 /home/adetayo/Documents/CSCI Forms/Adetayo Research/Cancer Screening Behavior/new_results/publication/new_results/processed/final_dataset.csv


In [18]:
# Screening summaries (Pap/Mammogram) from pre-negative CSV → results/ (+ trend 50–80)
# Build Pap/Mammogram summary tables (keeps negative codes) + trend plot (y=50–80)
# No-unzip: generate screening summaries from pre-negative CSV → results/
# Screening summary & trend (Yes%): preserve negative codes, export to results/
# Pap & Mammogram: summary tables + trend (50–80), using pre-negative CSV

# Heading: No-unzip — Screening summaries from pre-negative CSV → new_results/results (y=50–80)

import os, re
import matplotlib.pyplot as plt

# --- Paths (outputs to new_results/results/*) ---
base_path = r"D:\MamaZee\EB2_NIW\Publications\paper_1"
pre_negative_csv = os.path.join(base_path, "Preprocessing_Survey_Table", "nlsy_data_with_renamed_columns.csv")

results_root   = os.path.join(base_path, "new_results", "results")
results_tables = os.path.join(results_root, "tables")
results_figs   = os.path.join(results_root, "figs")
os.makedirs(results_tables, exist_ok=True)
os.makedirs(results_figs, exist_ok=True)

# --- Load pre-negative data (contains negative survey codes) ---
df = pd.read_csv(pre_negative_csv, low_memory=False)

# --- Years / columns (only keep existing) ---
years = ["2008", "2010", "2012", "2014", "2016", "2018"]
pap_cols = [c for c in [f"pap_smear_{y}" for y in years] if c in df.columns]
mam_cols = [c for c in [f"mammogram_{y}" for y in years] if c in df.columns]

# --- Summary function (keeps negatives; % within valid Yes+No) ---
def summarize_screening(data, cols):
    out = {
        "Response": [
            "Yes (1)", "No (0)", "Subtotal (Yes+No)",
            "Refusal (-1)", "Don't know (-2)", "Valid skip (-4)", "Non-interview (-5)",
            "Total (All)"
        ]
    }
    for col in cols:
        vc   = data[col].value_counts(dropna=False)
        c1   = int(vc.get(1, 0)); c0 = int(vc.get(0, 0))
        cm1  = int(vc.get(-1,0)); cm2= int(vc.get(-2,0))
        cm4  = int(vc.get(-4,0)); cm5= int(vc.get(-5,0))
        valid = c1 + c0
        total = c1 + c0 + cm1 + cm2 + cm4 + cm5  # exclude NA bucket
        ypct = f"{(c1/valid*100):.2f}%" if valid>0 else "0.00%"
        npct = f"{(c0/valid*100):.2f}%" if valid>0 else "0.00%"
        out[col] = [f"{c1} ({ypct})", f"{c0} ({npct})", str(valid),
                    str(cm1), str(cm2), str(cm4), str(cm5), str(total)]
    return pd.DataFrame(out)

pap_summary = summarize_screening(df, pap_cols) if pap_cols else pd.DataFrame()
mam_summary = summarize_screening(df, mam_cols) if mam_cols else pd.DataFrame()

# --- Save summaries to new_results/results/tables ---
if not pap_summary.empty:
    pap_path = os.path.join(results_tables, "pap_smear_summary_with_subtotals.csv")
    pap_summary.to_csv(pap_path, index=False)
    print("Saved:", pap_path)

if not mam_summary.empty:
    mam_path = os.path.join(results_tables, "mammogram_summary_with_subtotals.csv")
    mam_summary.to_csv(mam_path, index=False)
    print("Saved:", mam_path)

# --- Optional: save subset (ID + screening columns) to tables ---
subset_cols = (["id_code"] if "id_code" in df.columns else []) + pap_cols + mam_cols
if subset_cols:
    subset_out = os.path.join(results_tables, "pap_mammogram_2008_2018_subset.csv")
    df[subset_cols].to_csv(subset_out, index=False)
    print("Saved:", subset_out)

# --- Build trend table (Yes% only) from summaries ---
def extract_percent(s):
    if isinstance(s, str) and "(" in s:
        return float(s.split("(")[-1].replace(")", "").replace("%",""))
    return np.nan

def trend_from_summary(summary_df, label_prefix):
    if summary_df.empty: 
        return pd.DataFrame(columns=["Year", f"{label_prefix} (%)"])
    yes_row = summary_df.iloc[0]  # "Yes (1)"
    cols = [c for c in summary_df.columns if c != "Response"]
    yrs  = [re.search(r"(\d{4})", c).group(1) if re.search(r"(\d{4})", c) else c for c in cols]
    vals = [extract_percent(yes_row[c]) for c in cols]
    return pd.DataFrame({"Year": yrs, f"{label_prefix} (%)": vals})

pap_trend = trend_from_summary(pap_summary, "Pap Smear")
mam_trend = trend_from_summary(mam_summary, "Mammogram")

trend_df = pd.merge(pap_trend, mam_trend, on="Year", how="outer").sort_values("Year")
trend_out = os.path.join(results_tables, "screening_trend_table.csv")
trend_df.to_csv(trend_out, index=False)
print("Saved:", trend_out)

# --- Plot (y-axis 50..80, tick every 5) to new_results/results/figs ---
plt.figure(figsize=(8,5))
if not pap_trend.empty:
    plt.plot(pap_trend["Year"].astype(int), pap_trend["Pap Smear (%)"], marker='o', linewidth=2, label="Pap Smear (Yes %)")
if not mam_trend.empty:
    plt.plot(mam_trend["Year"].astype(int), mam_trend["Mammogram (%)"], marker='s', linewidth=2, label="Mammogram (Yes %)")

xticks = sorted(pd.to_numeric(trend_df["Year"], errors="coerce").dropna().astype(int).unique().tolist())
if xticks: plt.xticks(xticks)

plt.xlabel("Survey Year", fontsize=12)
plt.ylabel("Screening Rate (%)", fontsize=12)
plt.title("Screening Uptake Trends Over Time", fontsize=14)
plt.ylim(50, 80)
plt.yticks(list(range(50, 81, 5)))
plt.grid(linestyle="--", alpha=0.6)
plt.legend()
plt.tight_layout()

fig_out = os.path.join(results_figs, "appendix_trends.png")
plt.savefig(fig_out, dpi=300, bbox_inches='tight')
print("Saved:", fig_out)




FileNotFoundError: [Errno 2] No such file or directory: 'D:\\MamaZee\\EB2_NIW\\Publications\\paper_1/Preprocessing_Survey_Table/nlsy_data_with_renamed_columns.csv'

In [ ]:
# === XGBoost with RandomizedSearchCV (No SMOTE) – Year-based Split ===
# THIS IS IT!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

# === Load and sort panel data ===
df = pd.read_csv("final_dataset.csv")
df = df.sort_values(by=["id_code", "year"])

# === Forecast target and engineered features ===
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)
df['rolling_mean_lag1_3'] = df.groupby('id_code')['pap_smear'].shift(1).rolling(3).mean().reset_index(level=0, drop=True).fillna(0)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = df.groupby('id_code')['self_assement50'].expanding().mean().reset_index(level=0, drop=True)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# === Define features ===
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]
non_time_varying = ['race', 'educ', 'mother_educ', 'self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# === Train/Test split by year ===
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📆 Forecasting for {test_year} → predicting 2018")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
print(f"📏 Test observations for evaluation: {len(test_df)} rows")

# === Extract X and y ===
X_train = train_df[features]
y_train = train_df[target].astype(int)
X_test = test_df[features]
y_test = test_df[target].astype(int)

# === Scale features ===
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# === Define base model ===
xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42)

# === Define hyperparameter grid for RandomizedSearchCV ===
param_dist = {
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.3],
    'learning_rate': [0.01, 0.05, 0.1]
}

# === Randomized Search CV ===
print("\n🔍 Running RandomizedSearchCV for XGBoost...")
rand_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=10,
    scoring='roc_auc',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

rand_search.fit(X_train_scaled, y_train)

print(f"\n✅ Best Parameters from Search:")
print(rand_search.best_params_)

# === Retrain model with best parameters ===
best_xgb = rand_search.best_estimator_
best_xgb.fit(X_train_scaled, y_train)

# === Compute approximate "parameter count" for XGBoost ===
xgb_params = best_xgb.get_xgb_params()
n_trees = xgb_params.get("n_estimators", 100)
max_depth = xgb_params.get("max_depth", 3)
splits_per_tree = (2 ** max_depth) - 1
leaves_per_tree = (2 ** max_depth)
total_splits = n_trees * splits_per_tree
total_leaves = n_trees * leaves_per_tree
approx_params = total_splits + total_leaves

print("\n📊 XGBoost Model Complexity:")
print(f"➡ Number of Trees (n_estimators): {n_trees}")
print(f"➡ Max Depth per Tree: {max_depth}")
print(f"➡ Approx. Total Splits: {total_splits}")
print(f"➡ Approx. Total Leaves: {total_leaves}")
print(f"➡ 🔢 Approx. Total 'Parameters': {approx_params}")

# === Predict and Evaluate ===
y_pred_probs = best_xgb.predict_proba(X_test_scaled)[:, 1]
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# === AUC ===
auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 Final ROC AUC Score: {auc_score:.4f}")

fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()

# === SHAP Explainability ===
print("\n⚡ Running SHAP Analysis...")
explainer = shap.TreeExplainer(best_xgb)
shap_values = explainer.shap_values(X_test_scaled)

# === Rename SHAP features for interpretability ===
feature_name_map = {
    "pap_smear_lag1": "Lagged Pap Smear",
    "rolling_mean_lag1_3": "3-Round Screening Momentum",
    "income_log_pap_smear_lag1": "Log Income (Lag 1)",
    "race": "Race",
    "time_step": "Time Step",
    "health_facility": "Health Facility Access",
    "educ_mother_educ": "Mother's Education",
    "income_log": "Log Income",
    "income_log_self60": "Income (Age 60)",
    "trend_income_log": "Income Trend",
    "region_year": "Region-Year Interaction",
    "income_log_region": "Regional Income",
    "health_plan": "Health Plan",
    "year": "Survey Year",
    "income_log_self50": "Income (Age 50)",
    "mother_educ": "Mother’s Education",
    "educ": "Education",
    "hh_children": "Children in Household",
    "self_assement50": "Self-Health Rating (Age 50)",
    "income_log_health_provider": "Income-Health Provider Interaction",
    "marital_status": "Marital Status",
    "region": "Region",
    "health_provider": "Health Provider",
    "race_health_plan": "Race-Health Plan Interaction",
    "region_health_plan": "Region-Health Plan Interaction"
}

X_test_renamed = X_test.copy()
X_test_renamed.columns = [feature_name_map.get(col, col) for col in X_test.columns]

# === SHAP Bar Plot with readable feature names ===
shap.summary_plot(shap_values, X_test_renamed, plot_type='bar')

# === SHAP Beeswarm Plot with readable feature names ===
shap.summary_plot(shap_values, X_test_renamed)


In [ ]:
# combines extracted SHAP plots from mammogram and pap smear from xgboost 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Load the fixed SHAP plots
pap_path = r'D:/MamaZee/EB2_NIW/Publications/paper_1/new_results/results/figs/pap smear SHAP.png'
mammo_path = r'D:/MamaZee/EB2_NIW/Publications/paper_1/new_results/results/figs/mammogram SHAP fixed.png'

pap_img = mpimg.imread(pap_path)
mammo_img = mpimg.imread(mammo_path)

fig, axs = plt.subplots(1, 2, figsize=(20, 12))

axs[0].imshow(pap_img)
axs[0].axis('off')
axs[0].set_title("Pap Smear SHAP", fontsize=18)

axs[1].imshow(mammo_img)
axs[1].axis('off')
axs[1].set_title("Mammogram SHAP", fontsize=18)

plt.subplots_adjust(left=0.01, right=0.99, top=0.92, bottom=0.08, wspace=0.05)

save_path = r'D:/MamaZee/EB2_NIW/Publications/paper_1/new_results/results/figs/shap_combined_fixed.png'
plt.savefig(save_path, dpi=300)
plt.show()

print(f"✅ Combined image saved: {save_path}")


In [ ]:
# ✅ LSTM for Pap Smear Forecasting (No SMOTE, Locked Hyperparameters, Class Weights)
# 🚫 SMOTE removed — LSTM for Pap Smear Forecasting
# No Embeddings
# Goal: AUC ~ 0.75 consistently
# Final 

# 1 LSTM layer (64 units)
# 1 Dropout layer (30% rate)
# 1 Dense layer (sigmoid activation for binary classification)
# Learning rate: 0.001
# Dropout(0.3)

# Use this one:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("final_dataset.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)
df['rolling_mean_lag1_3'] = df.groupby('id_code')['pap_smear'].shift(1).rolling(3).mean().reset_index(level=0, drop=True).fillna(0)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = df.groupby('id_code')['self_assement50'].expanding().mean().reset_index(level=0, drop=True)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 🧮 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]
non_time_varying = ['race', 'educ', 'mother_educ', 'self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 📅 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
y_train = train_df[target].astype(int)
X_test = test_df[features]
y_test = test_df[target].astype(int)

# ===========================
# ⚖️ Class weights instead of SMOTE
# ===========================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print(f"\n⚖️ Class Weights: {class_weight_dict}")

# ===========================
# 🔍 Scale
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔁 Reshape for LSTM
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Locked Hyperparameters for Stability
# ===========================
model = Sequential([
    LSTM(64, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


# === Print model summary and extract total params ===
print("\n📊 LSTM Model Summary:")
model.summary()

# Capture the total number of trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 🛑 Early stopping
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ===========================
# 🚀 Train model (No SMOTE, With Class Weights)
# ===========================
model.fit(
    X_train_lstm, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_lstm, y_test),
    callbacks=[early_stop],
    class_weight=class_weight_dict,
    verbose=1
)

# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict(X_test_lstm).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ LSTM (Locked Hyperparameters) + Embeddings for Time-Invariant Vars (race, educ, mother_educ)
# Final for LSTM 

# Embedding dim: 4
# LSTM units: 128
# Dropout: 0.1
# Learning rate: 0.001

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("final_dataset.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding (NO id_code embedding)
# ===========================
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)

df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['pap_smear'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)

df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]

# Only non-time-varying features that are NOT embedded
non_time_varying = ['self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for LSTM (1 timestep)
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build FINAL LSTM model (LOCKED hyperparameters)
# ===========================
num_input = Input(shape=(1, X_train_lstm.shape[2]), name='num_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# ✅ Locked embedding dim = 4
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine inputs
combined_input = Concatenate()([num_input, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded])

# ✅ Locked LSTM units = 128, Dropout = 0.1
x = LSTM(128)(combined_input)
x = Dropout(0.1)(x)

# Output
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, race_input, educ_input, meduc_input], outputs=output)

# ✅ Locked learning rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


# === Print model summary and extract total params ===
print("\n📊 LSTM + Static Embeddings Model Summary:")
model.summary()

# Count total trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 🚀 Train FINAL model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_lstm, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_lstm, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)

# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_lstm, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ LSTM (Locked Params) + Super-Regularized ID Embedding
# Goal: ID embedding too weak to memorize, AUC should drop closer to 0.87–0.89
# Final 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    LSTM, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector,
    Flatten, GaussianNoise
)
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("final_dataset.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding
# ===========================
df['id_code_int'] = df['id_code'].astype('category').cat.codes
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_ids = df['id_code_int'].nunique()
num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)
df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['pap_smear'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]
non_time_varying = ['self_assement50', 'self_assement60']  # race/educ/mother_educ embedded separately

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_id = train_df['id_code_int'].values
X_test_id = test_df['id_code_int'].values
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for LSTM
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build LSTM model (Locked Params + Weak ID Embedding)
# ===========================
num_input = Input(shape=(1, X_train_lstm.shape[2]), name='num_input')
id_input = Input(shape=(1,), name='id_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# 🔒 ID embedding (super regularized)
id_embed = Embedding(
    input_dim=num_ids,
    output_dim=1,                             # just scalar
    embeddings_regularizer=tf.keras.regularizers.l2(1e-2)
)(id_input)
id_embed = GaussianNoise(0.3)(id_embed)       # add strong noise
id_embed = Dropout(0.7)(id_embed)             # very heavy dropout
id_embed_flat = Flatten()(id_embed)
id_embed_expanded = RepeatVector(1)(id_embed_flat)

# 🔒 Time-invariant embeddings (locked)
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand for time-step dimension
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine numeric + embeddings
combined_input = Concatenate()([
    num_input, id_embed_expanded, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded
])

# 🔒 LSTM (locked)
x = LSTM(128)(combined_input)
x = Dropout(0.1)(x)

# Output layer
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, id_input, race_input, educ_input, meduc_input], outputs=output)

# 🔒 Locked learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# === Print model summary and count parameters ===
print("\n📊 LSTM + ID Embedding Model Summary:")
model.summary()

# Count total trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 🚀 Train Model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_lstm, X_train_id, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_lstm, X_test_id, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)

# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_lstm, X_test_id, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# ROC-AUC
auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ Bi-LSTM for Pap Smear Forecasting (No SMOTE, Locked Hyperparameters, Class Weights)
# 🚫 SMOTE removed — LSTM for Pap Smear Forecasting
# No Embeddings
# AUC ~ 0.7703 consistently

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("final_dataset.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)
df['rolling_mean_lag1_3'] = df.groupby('id_code')['pap_smear'].shift(1).rolling(3).mean().reset_index(level=0, drop=True).fillna(0)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = df.groupby('id_code')['self_assement50'].expanding().mean().reset_index(level=0, drop=True)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 🧮 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]
non_time_varying = ['race', 'educ', 'mother_educ', 'self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 📅 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
y_train = train_df[target].astype(int)
X_test = test_df[features]
y_test = test_df[target].astype(int)

# ===========================
# ⚖️ Class weights instead of SMOTE
# ===========================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print(f"\n⚖️ Class Weights: {class_weight_dict}")

# ===========================
# 🔍 Scale
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔁 Reshape for LSTM
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 BiLSTM Model (Locked Hyperparameters for Stability)
# ===========================
model = Sequential([
    Bidirectional(LSTM(64, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]))),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🛑 Early stopping
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ===========================
# 🚀 Train model (No SMOTE, With Class Weights)
# ===========================
model.fit(
    X_train_lstm, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_lstm, y_test),
    callbacks=[early_stop],
    class_weight=class_weight_dict,
    verbose=1
)

# ===========================
# 📊 Print Model Summary and Count Params (AFTER training)
# ===========================
print("\n📊 Bi-LSTM Model Summary:")
model.summary()

trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")

# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict(X_test_lstm).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ BiLSTM (Locked Hyperparameters) + Embeddings for Time-Invariant Vars (race, educ, mother_educ)
# Same locked params: Embedding dim = 4, LSTM units = 128, Dropout = 0.1, Learning rate = 0.001
# Final ~ 0.91
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    LSTM, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector, Flatten, Bidirectional
)
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("final_dataset.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding (NO id_code embedding)
# ===========================
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)

df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['pap_smear'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)

df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]

# Only non-time-varying features that are NOT embedded
non_time_varying = ['self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for BiLSTM (1 timestep)
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build FINAL BiLSTM model (LOCKED hyperparameters)
# ===========================
num_input = Input(shape=(1, X_train_lstm.shape[2]), name='num_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# ✅ Locked embedding dim = 4
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine inputs
combined_input = Concatenate()([num_input, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded])

# ✅ BiLSTM instead of single LSTM
x = Bidirectional(LSTM(128))(combined_input)
x = Dropout(0.1)(x)

# Output layer
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, race_input, educ_input, meduc_input], outputs=output)

# ✅ Locked learning rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train FINAL BiLSTM model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_lstm, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_lstm, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)


# ===========================
# 📊 Print Model Summary & Count Params
# ===========================
print("\n📊 BiLSTM + Static Embeddings Model Summary:")
model.summary()

# Count total trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")



# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_lstm, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ BiLSTM (Locked Params) + Super-Regularized ID Embedding
# Goal: ID embedding is weak (can’t memorize individuals), AUC closer to ~0.87–0.89
# Final: 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    LSTM, Bidirectional, Dense, Dropout, Embedding, Input,
    Concatenate, RepeatVector, Flatten, GaussianNoise
)
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("final_dataset.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding
# ===========================
df['id_code_int'] = df['id_code'].astype('category').cat.codes
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_ids = df['id_code_int'].nunique()
num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)
df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['pap_smear'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]
non_time_varying = ['self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_id = train_df['id_code_int'].values
X_test_id = test_df['id_code_int'].values
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for BiLSTM
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build BiLSTM model (Locked Params + Weak ID Embedding)
# ===========================
num_input = Input(shape=(1, X_train_lstm.shape[2]), name='num_input')
id_input = Input(shape=(1,), name='id_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# 🔒 ID embedding (super regularized)
id_embed = Embedding(
    input_dim=num_ids,
    output_dim=1,                            # scalar embedding
    embeddings_regularizer=tf.keras.regularizers.l2(1e-2)
)(id_input)
id_embed = GaussianNoise(0.3)(id_embed)      # strong noise
id_embed = Dropout(0.7)(id_embed)            # very heavy dropout
id_embed_flat = Flatten()(id_embed)
id_embed_expanded = RepeatVector(1)(id_embed_flat)

# 🔒 Time-invariant embeddings
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand for timestep dimension
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine numeric + embeddings
combined_input = Concatenate()([
    num_input, id_embed_expanded, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded
])

# 🔄 Replace LSTM with BiLSTM (still 128 units)
x = Bidirectional(LSTM(128))(combined_input)
x = Dropout(0.1)(x)

# Output
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, id_input, race_input, educ_input, meduc_input], outputs=output)

# 🔒 Locked learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train BiLSTM Model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_lstm, X_train_id, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_lstm, X_test_id, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)


# ===========================
# 📊 Print Model Summary & Count Parameters
# ===========================
print("\n📊 BiLSTM + Super-Regularized ID Embedding Model Summary:")
model.summary()

# Count trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_lstm, X_test_id, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# ROC-AUC
auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU for Pap Smear Forecasting (No SMOTE, Locked Hyperparameters, Class Weights)
# 🚫 SMOTE removed — GRU version
# No Embeddings
# AUC ~ 0.75 consistently

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("final_dataset.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)
df['rolling_mean_lag1_3'] = df.groupby('id_code')['pap_smear'].shift(1).rolling(3).mean().reset_index(level=0, drop=True).fillna(0)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = df.groupby('id_code')['self_assement50'].expanding().mean().reset_index(level=0, drop=True)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 🧮 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]
non_time_varying = ['race', 'educ', 'mother_educ', 'self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 📅 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
y_train = train_df[target].astype(int)
X_test = test_df[features]
y_test = test_df[target].astype(int)

# ===========================
# ⚖️ Class weights instead of SMOTE
# ===========================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print(f"\n⚖️ Class Weights: {class_weight_dict}")

# ===========================
# 🔍 Scale
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔁 Reshape for GRU (same shape as LSTM)
X_train_gru = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_gru = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 GRU Model (Locked Hyperparameters)
# ===========================
model = Sequential([
    GRU(64, input_shape=(X_train_gru.shape[1], X_train_gru.shape[2])),   # ✅ GRU instead of Bi-LSTM
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🛑 Early stopping
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ===========================
# 🚀 Train model (No SMOTE, With Class Weights)
# ===========================
model.fit(
    X_train_gru, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_gru, y_test),
    callbacks=[early_stop],
    class_weight=class_weight_dict,
    verbose=1
)


# ===========================
# 📊 Print Model Summary & Count Parameters
# ===========================
print("\n📊 GRU Model Summary:")
model.summary()

# Count trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict(X_test_gru).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU (Locked Hyperparameters) + Embeddings for Time-Invariant Vars (race, educ, mother_educ)
# Same settings as your LSTM version:
# - Embedding dim: 4
# - GRU units: 128 (same as LSTM units)
# - Dropout: 0.1
# - Learning rate: 0.001

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("final_dataset.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding (NO id_code embedding)
# ===========================
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)

df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['pap_smear'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)

df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]

# Only non-time-varying features that are NOT embedded
non_time_varying = ['self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for GRU (1 timestep)
X_train_gru = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_gru = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build FINAL GRU model (LOCKED hyperparameters)
# ===========================
num_input = Input(shape=(1, X_train_gru.shape[2]), name='num_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# ✅ Locked embedding dim = 4
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine inputs
combined_input = Concatenate()([num_input, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded])

# ✅ GRU instead of LSTM
x = GRU(128)(combined_input)
x = Dropout(0.1)(x)

# Output
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, race_input, educ_input, meduc_input], outputs=output)

# ✅ Locked learning rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train FINAL model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_gru, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_gru, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)


# ===========================
# 📊 Print Model Summary & Count Parameters
# ===========================
print("\n📊 GRU + Static Embeddings Model Summary:")
model.summary()

# Count total trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")



# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_gru, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU (Locked Params) + Super-Regularized ID Embedding
# Goal: ID embedding too weak to memorize, AUC should drop closer to 0.87–0.89

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    GRU, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector,
    Flatten, GaussianNoise
)
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("final_dataset.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding
# ===========================
df['id_code_int'] = df['id_code'].astype('category').cat.codes
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_ids = df['id_code_int'].nunique()
num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)
df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['pap_smear'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]
non_time_varying = ['self_assement50', 'self_assement60']  # race/educ/mother_educ embedded separately

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_id = train_df['id_code_int'].values
X_test_id = test_df['id_code_int'].values
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for GRU
X_train_gru = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_gru = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build GRU model (Locked Params + Weak ID Embedding)
# ===========================
num_input = Input(shape=(1, X_train_gru.shape[2]), name='num_input')
id_input = Input(shape=(1,), name='id_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# 🔒 ID embedding (super regularized)
id_embed = Embedding(
    input_dim=num_ids,
    output_dim=1,                             # just scalar
    embeddings_regularizer=tf.keras.regularizers.l2(1e-2)
)(id_input)
id_embed = GaussianNoise(0.3)(id_embed)       # add strong noise
id_embed = Dropout(0.7)(id_embed)             # very heavy dropout
id_embed_flat = Flatten()(id_embed)
id_embed_expanded = RepeatVector(1)(id_embed_flat)

# 🔒 Time-invariant embeddings (locked)
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand for time-step dimension
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine numeric + embeddings
combined_input = Concatenate()([
    num_input, id_embed_expanded, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded
])

# 🔒 GRU (instead of LSTM)
x = GRU(128)(combined_input)
x = Dropout(0.1)(x)

# Output layer
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, id_input, race_input, educ_input, meduc_input], outputs=output)

# 🔒 Locked learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train Model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_gru, X_train_id, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_gru, X_test_id, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)

# ===========================
# 📊 Print Model Summary & Count Parameters
# ===========================
print("\n📊 GRU + Super-Regularized ID Embedding Model Summary:")
model.summary()

# Count trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_gru, X_test_id, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# ROC-AUC
auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU-D for Pap Smear Forecasting (No SMOTE, Locked Hyperparameters, Class Weights)
# 🚫 SMOTE removed — GRU-D version
# No Embeddings
# Handles Missingness Explicitly
# auc ~ 0.75

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Dense, Dropout, Input, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("final_dataset.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)
df['rolling_mean_lag1_3'] = df.groupby('id_code')['pap_smear'].shift(1).rolling(3).mean().reset_index(level=0, drop=True).fillna(0)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = df.groupby('id_code')['self_assement50'].expanding().mean().reset_index(level=0, drop=True)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 🧮 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]
non_time_varying = ['race', 'educ', 'mother_educ', 'self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 📅 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=[target])  # ✅ Only drop rows missing the target
test_df = test_df.dropna(subset=[target])
print(f"After dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y and ensure float conversion
# ===========================
X_train = train_df[features].astype(float).values
y_train = train_df[target].astype(int)
X_test = test_df[features].astype(float).values
y_test = test_df[target].astype(int)

# ⚖️ Class weights instead of SMOTE
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print(f"\n⚖️ Class Weights: {class_weight_dict}")

# ===========================
# 🔍 Handle Missingness for GRU-D
# ===========================
# 1️⃣ Create masks (1 if present, 0 if missing)
mask_train = np.where(np.isnan(X_train), 0, 1).astype(float)
mask_test = np.where(np.isnan(X_test), 0, 1).astype(float)

# 2️⃣ Fill missing with zero for GRU-D (model learns decay)
X_train_filled = np.nan_to_num(X_train, nan=0)
X_test_filled = np.nan_to_num(X_test, nan=0)

# ===========================
# 🔍 Scale numeric inputs (after filling)
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filled)
X_test_scaled = scaler.transform(X_test_filled)

# 🔁 Reshape for GRU (sequence length = 1 timestep)
X_train_seq = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_seq = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

mask_train_seq = mask_train.reshape((mask_train.shape[0], 1, mask_train.shape[1]))
mask_test_seq = mask_test.reshape((mask_test.shape[0], 1, mask_test.shape[1]))

# ===========================
# 🏗 GRU-D Model (Simplified version of GRU-D)
# ===========================
num_input = Input(shape=(1, X_train_seq.shape[2]), name='num_input')
mask_input = Input(shape=(1, X_train_seq.shape[2]), name='mask_input')

# ✅ GRU takes both numeric values and mask concatenated
combined_input = Concatenate(axis=-1)([num_input, mask_input])

x = GRU(64)(combined_input)   # 🔒 Locked units
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[num_input, mask_input], outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🛑 Early stopping
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ===========================
# 🚀 Train model (No SMOTE, With Class Weights)
# ===========================
model.fit(
    [X_train_seq, mask_train_seq], y_train,
    epochs=50,
    batch_size=32,
    validation_data=([X_test_seq, mask_test_seq], y_test),
    callbacks=[early_stop],
    class_weight=class_weight_dict,
    verbose=1
)

# ===========================
# 📊 Print Model Summary & Count Trainable Parameters
# ===========================
print("\n📊 GRU-D Model Summary:")
model.summary()

# Count trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_seq, mask_test_seq]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU-D (Locked Hyperparameters) + Embeddings for Time-Invariant Vars (race, educ, mother_educ)
# Handles missingness with masks and decay terms (GRU-D style)
# Embedding dim: 4 | GRU units: 128 | Dropout: 0.1 | LR: 0.001

# AUC ~ 0.85

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector, Flatten, Multiply
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding (NO id_code embedding)
# ===========================
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)
df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['pap_smear'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]

non_time_varying = ['self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 🔍 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ===========================
# 🧩 GRU-D style missingness handling
# ===========================
# ✅ 1. Create masks (1 if present, 0 if missing) → convert to NumPy!
mask_train = (~X_train.isna()).astype(float).values
mask_test = (~X_test.isna()).astype(float).values

# ✅ 2. Replace NaNs with 0 (GRU-D learns to use mask for missing info)
X_train_filled = np.nan_to_num(X_train_scaled, nan=0)
X_test_filled = np.nan_to_num(X_test_scaled, nan=0)

# ✅ 3. Reshape for GRU input
X_train_gru = X_train_filled.reshape((X_train_filled.shape[0], 1, X_train_filled.shape[1]))
X_test_gru = X_test_filled.reshape((X_test_filled.shape[0], 1, X_test_filled.shape[1]))

mask_train_gru = mask_train.reshape((mask_train.shape[0], 1, mask_train.shape[1]))
mask_test_gru = mask_test.reshape((mask_test.shape[0], 1, mask_test.shape[1]))

# ===========================
# 🏗 GRU-D Model (Locked Hyperparameters)
# ===========================
num_input = Input(shape=(1, X_train_gru.shape[2]), name='num_input')
mask_input = Input(shape=(1, X_train_gru.shape[2]), name='mask_input')

# ✅ Apply mask to input (GRU-D style)
masked_input = Multiply()([num_input, mask_input])

race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# ✅ Locked embedding dim = 4
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand for time-step dimension
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# ✅ Combine everything
combined_input = Concatenate()([masked_input, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded])

# ✅ GRU layer (128 units, dropout = 0.1)
x = GRU(128)(combined_input)
x = Dropout(0.1)(x)

# Output layer
output = Dense(1, activation='sigmoid')(x)

# ✅ Build GRU-D model
model = Model(inputs=[num_input, mask_input, race_input, educ_input, meduc_input], outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train GRU-D Model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_gru, mask_train_gru, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_gru, mask_test_gru, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)


# ===========================
# 📊 Print Model Summary & Count Trainable Parameters
# ===========================
print("\n📊 GRU-D + Static Embeddings Model Summary:")
model.summary()

# Count trainable parameters for reporting in your table
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")



# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_gru, mask_test_gru, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU-D (Locked Params) + Super-Regularized ID Embedding
# Goal: GRU-D handles missingness with masks; ID embedding heavily regularized
# Expected: ID embedding too weak to memorize, AUC should drop closer to 0.87–0.89

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    GRU, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector,
    Flatten, GaussianNoise, Multiply
)
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding
# ===========================
df['id_code_int'] = df['id_code'].astype('category').cat.codes
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_ids = df['id_code_int'].nunique()
num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['pap_smear_t_plus_1'] = df.groupby('id_code')['pap_smear'].shift(-1)
df['pap_smear_lag1'] = df.groupby('id_code')['pap_smear'].shift(1)
df['pap_smear_lag2'] = df.groupby('id_code')['pap_smear'].shift(2)
df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['pap_smear'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_pap_smear_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'pap_smear_lag1'
]
non_time_varying = ['self_assement50', 'self_assement60']  # race/educ/mother_educ embedded separately

features = time_varying + non_time_varying
target = 'pap_smear_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014]
test_year = 2016

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_id = train_df['id_code_int'].values
X_test_id = test_df['id_code_int'].values
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ===========================
# 🧩 GRU-D style missingness handling
# ===========================
mask_train = (~X_train.isna()).astype(float).values
mask_test = (~X_test.isna()).astype(float).values

X_train_filled = np.nan_to_num(X_train_scaled, nan=0)
X_test_filled = np.nan_to_num(X_test_scaled, nan=0)

# Reshape for GRU (and mask)
X_train_gru = X_train_filled.reshape((X_train_filled.shape[0], 1, X_train_filled.shape[1]))
X_test_gru = X_test_filled.reshape((X_test_filled.shape[0], 1, X_test_filled.shape[1]))

mask_train_gru = mask_train.reshape((mask_train.shape[0], 1, mask_train.shape[1]))
mask_test_gru = mask_test.reshape((mask_test.shape[0], 1, mask_test.shape[1]))

# ===========================
# 🏗 Build GRU-D model (Locked Params + Weak ID Embedding)
# ===========================
num_input = Input(shape=(1, X_train_gru.shape[2]), name='num_input')
mask_input = Input(shape=(1, X_train_gru.shape[2]), name='mask_input')

# ✅ GRU-D style: apply mask to input
masked_input = Multiply()([num_input, mask_input])

# ID + other embeddings
id_input = Input(shape=(1,), name='id_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# 🔒 ID embedding (super regularized)
id_embed = Embedding(input_dim=num_ids, output_dim=1,
                     embeddings_regularizer=tf.keras.regularizers.l2(1e-2))(id_input)
id_embed = GaussianNoise(0.3)(id_embed)
id_embed = Dropout(0.7)(id_embed)
id_embed_flat = Flatten()(id_embed)
id_embed_expanded = RepeatVector(1)(id_embed_flat)

# 🔒 Time-invariant embeddings
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand embeddings
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# ✅ Combine numeric, mask-handled input, and embeddings
combined_input = Concatenate()([
    masked_input, id_embed_expanded, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded
])

# ✅ GRU (locked params)
x = GRU(128)(combined_input)
x = Dropout(0.1)(x)

# Output layer
output = Dense(1, activation='sigmoid')(x)

# ✅ Build GRU-D model
model = Model(inputs=[num_input, mask_input, id_input, race_input, educ_input, meduc_input],
              outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train Model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_gru, mask_train_gru, X_train_id, X_train_race, X_train_educ, X_train_meduc],
          y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_gru, mask_test_gru, X_test_id, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)

# ✅ Print model summary and count parameters
model.summary()

total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {total_params}")


# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_gru, mask_test_gru, X_test_id, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# ROC-AUC
auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Mammogram 

In [ ]:
# === XGBoost with RandomizedSearchCV (No SMOTE) – Year-based Split ===
# THIS IS IT!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

# === Load and sort panel data ===
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# === Forecast target and engineered features ===
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)
df['rolling_mean_lag1_3'] = df.groupby('id_code')['mammogram'].shift(1).rolling(3).mean().reset_index(level=0, drop=True).fillna(0)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = df.groupby('id_code')['self_assement50'].expanding().mean().reset_index(level=0, drop=True)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)


# === Define features ===
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]
non_time_varying = ['race', 'educ', 'mother_educ', 'self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# === Train/Test split by year ===
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📆 Forecasting for {test_year} → predicting 2018")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
print(f"📏 Test observations for evaluation: {len(test_df)} rows")

# === Extract X and y ===
X_train = train_df[features]
y_train = train_df[target].astype(int)
X_test = test_df[features]
y_test = test_df[target].astype(int)

# === Scale features ===
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# === Define base model ===
xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42)

# === Define hyperparameter grid for RandomizedSearchCV ===
param_dist = {
    'max_depth': [3, 4, 5, 6],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.3],
    'learning_rate': [0.01, 0.05, 0.1]
}

# === Randomized Search CV ===
print("\n🔍 Running RandomizedSearchCV for XGBoost...")
rand_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=10,   # 🔟 TRIALS
    scoring='roc_auc',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

rand_search.fit(X_train_scaled, y_train)

print(f"\n✅ Best Parameters from Search:")
print(rand_search.best_params_)

# === Retrain model with best parameters ===
best_xgb = rand_search.best_estimator_
best_xgb.fit(X_train_scaled, y_train)

# === Compute approximate "parameter count" for XGBoost ===
xgb_params = best_xgb.get_xgb_params()

n_trees = xgb_params.get("n_estimators", 100)
max_depth = xgb_params.get("max_depth", 3)

if n_trees is None:
    n_trees = 100
if max_depth is None:
    max_depth = 3

# Each tree of depth d can have (2^d - 1) splits and (2^d) leaves
splits_per_tree = (2 ** max_depth) - 1
leaves_per_tree = (2 ** max_depth)

total_splits = n_trees * splits_per_tree
total_leaves = n_trees * leaves_per_tree
approx_params = total_splits + total_leaves

print("\n📊 XGBoost Model Complexity:")
print(f"➡ Number of Trees (n_estimators): {n_trees}")
print(f"➡ Max Depth per Tree: {max_depth}")
print(f"➡ Approx. Total Splits: {total_splits}")
print(f"➡ Approx. Total Leaves: {total_leaves}")
print(f"➡ 🔢 Approx. Total 'Parameters': {approx_params}")


# === Predict and Evaluate ===
y_pred_probs = best_xgb.predict_proba(X_test_scaled)[:, 1]
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# === AUC ===
auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 Final ROC AUC Score: {auc_score:.4f}")

fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()

# === SHAP Explainability ===
print("\n⚡ Running SHAP Analysis...")
explainer = shap.TreeExplainer(best_xgb)
shap_values = explainer.shap_values(X_test_scaled)

# === Rename SHAP features for interpretability ===
feature_name_map = {
    "mammogram_lag1": "Lagged Mammogram",
    "rolling_mean_lag1_3": "3-Round Screening Momentum",
    "income_log_mammogram_lag1": "Log Income (Lag 1)",
    "race": "Race",
    "time_step": "Time Step",
    "health_facility": "Health Facility Access",
    "educ_mother_educ": "Mother's Education",
    "income_log": "Log Income",
    "income_log_self60": "Income (Age 60)",
    "trend_income_log": "Income Trend",
    "region_year": "Region-Year Interaction",
    "income_log_region": "Regional Income",
    "health_plan": "Health Plan",
    "year": "Survey Year",
    "income_log_self50": "Income (Age 50)",
    "mother_educ": "Mother’s Education",
    "educ": "Education",
    "hh_children": "Children in Household",
    "self_assement50": "Self-Health Rating (Age 50)",
    "income_log_health_provider": "Income-Health Provider Interaction",
    "marital_status": "Marital Status",
    "region": "Region",
    "health_provider": "Health Provider",
    "race_health_plan": "Race-Health Plan Interaction",
    "region_health_plan": "Region-Health Plan Interaction"
}

X_test_renamed = X_test.copy()
X_test_renamed.columns = [feature_name_map.get(col, col) for col in X_test.columns]

# === SHAP Bar Plot with readable feature names ===
shap.summary_plot(shap_values, X_test_renamed, plot_type='bar')

# === SHAP Beeswarm Plot with readable feature names ===
shap.summary_plot(shap_values, X_test_renamed)


In [ ]:
# ✅ LSTM for mammogram Forecasting (No SMOTE, Locked Hyperparameters, Class Weights)
# 🚫 SMOTE removed — LSTM for Pap Smear Forecasting
# No Embeddings
# Goal: AUC ~ 0.75 consistently
# Final 

# 1 LSTM layer (64 units)
# 1 Dropout layer (30% rate)
# 1 Dense layer (sigmoid activation for binary classification)
# Learning rate: 0.001
# Dropout(0.3)

# Use this one:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)
df['rolling_mean_lag1_3'] = df.groupby('id_code')['mammogram'].shift(1).rolling(3).mean().reset_index(level=0, drop=True).fillna(0)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = df.groupby('id_code')['self_assement50'].expanding().mean().reset_index(level=0, drop=True)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)


# ===========================
# 🧮 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]
non_time_varying = ['race', 'educ', 'mother_educ', 'self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 📅 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
y_train = train_df[target].astype(int)
X_test = test_df[features]
y_test = test_df[target].astype(int)

# ===========================
# ⚖️ Class weights instead of SMOTE
# ===========================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print(f"\n⚖️ Class Weights: {class_weight_dict}")

# ===========================
# 🔍 Scale
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔁 Reshape for LSTM
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Locked Hyperparameters for Stability
# ===========================
model = Sequential([
    LSTM(64, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


# === Print model summary and extract total params ===
print("\n📊 LSTM Model Summary:")
model.summary()

# Capture the total number of trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 🛑 Early stopping
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ===========================
# 🚀 Train model (No SMOTE, With Class Weights)
# ===========================
model.fit(
    X_train_lstm, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_lstm, y_test),
    callbacks=[early_stop],
    class_weight=class_weight_dict,
    verbose=1
)

# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict(X_test_lstm).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ LSTM (Locked Hyperparameters) + Embeddings for Time-Invariant Vars (race, educ, mother_educ)
# Final for LSTM 

# Embedding dim: 4
# LSTM units: 128
# Dropout: 0.1
# Learning rate: 0.001

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding (NO id_code embedding)
# ===========================
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)

df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['mammogram'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)

df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]

# Only non-time-varying features that are NOT embedded
non_time_varying = ['self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for LSTM (1 timestep)
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build FINAL LSTM model (LOCKED hyperparameters)
# ===========================
num_input = Input(shape=(1, X_train_lstm.shape[2]), name='num_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# ✅ Locked embedding dim = 4
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine inputs
combined_input = Concatenate()([num_input, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded])

# ✅ Locked LSTM units = 128, Dropout = 0.1
x = LSTM(128)(combined_input)
x = Dropout(0.1)(x)

# Output
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, race_input, educ_input, meduc_input], outputs=output)

# ✅ Locked learning rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


# === Print model summary and extract total params ===
print("\n📊 LSTM + Static Embeddings Model Summary:")
model.summary()

# Count total trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 🚀 Train FINAL model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_lstm, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_lstm, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)

# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_lstm, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ LSTM (Locked Params) + Super-Regularized ID Embedding
# Goal: ID embedding too weak to memorize, AUC should drop closer to 0.87–0.89
# Final 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    LSTM, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector,
    Flatten, GaussianNoise
)
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding
# ===========================
df['id_code_int'] = df['id_code'].astype('category').cat.codes
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_ids = df['id_code_int'].nunique()
num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)
df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['mammogram'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)


# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]
non_time_varying = ['self_assement50', 'self_assement60']  # race/educ/mother_educ embedded separately

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_id = train_df['id_code_int'].values
X_test_id = test_df['id_code_int'].values
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for LSTM
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build LSTM model (Locked Params + Weak ID Embedding)
# ===========================
num_input = Input(shape=(1, X_train_lstm.shape[2]), name='num_input')
id_input = Input(shape=(1,), name='id_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# 🔒 ID embedding (super regularized)
id_embed = Embedding(
    input_dim=num_ids,
    output_dim=1,                             # just scalar
    embeddings_regularizer=tf.keras.regularizers.l2(1e-2)
)(id_input)
id_embed = GaussianNoise(0.3)(id_embed)       # add strong noise
id_embed = Dropout(0.7)(id_embed)             # very heavy dropout
id_embed_flat = Flatten()(id_embed)
id_embed_expanded = RepeatVector(1)(id_embed_flat)

# 🔒 Time-invariant embeddings (locked)
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand for time-step dimension
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine numeric + embeddings
combined_input = Concatenate()([
    num_input, id_embed_expanded, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded
])

# 🔒 LSTM (locked)
x = LSTM(128)(combined_input)
x = Dropout(0.1)(x)

# Output layer
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, id_input, race_input, educ_input, meduc_input], outputs=output)

# 🔒 Locked learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# === Print model summary and count parameters ===
print("\n📊 LSTM + ID Embedding Model Summary:")
model.summary()

# Count total trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 🚀 Train Model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_lstm, X_train_id, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_lstm, X_test_id, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)

# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_lstm, X_test_id, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# ROC-AUC
auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ Bi-LSTM for Pap Smear Forecasting (No SMOTE, Locked Hyperparameters, Class Weights)
# 🚫 SMOTE removed — LSTM for Pap Smear Forecasting
# No Embeddings
# AUC ~ 0.7703 consistently

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)
df['rolling_mean_lag1_3'] = df.groupby('id_code')['mammogram'].shift(1).rolling(3).mean().reset_index(level=0, drop=True).fillna(0)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = df.groupby('id_code')['self_assement50'].expanding().mean().reset_index(level=0, drop=True)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)

# ===========================
# 🧮 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]
non_time_varying = ['race', 'educ', 'mother_educ', 'self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 📅 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
y_train = train_df[target].astype(int)
X_test = test_df[features]
y_test = test_df[target].astype(int)

# ===========================
# ⚖️ Class weights instead of SMOTE
# ===========================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print(f"\n⚖️ Class Weights: {class_weight_dict}")

# ===========================
# 🔍 Scale
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔁 Reshape for LSTM
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 BiLSTM Model (Locked Hyperparameters for Stability)
# ===========================
model = Sequential([
    Bidirectional(LSTM(64, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]))),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🛑 Early stopping
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ===========================
# 🚀 Train model (No SMOTE, With Class Weights)
# ===========================
model.fit(
    X_train_lstm, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_lstm, y_test),
    callbacks=[early_stop],
    class_weight=class_weight_dict,
    verbose=1
)

# ===========================
# 📊 Print Model Summary and Count Params (AFTER training)
# ===========================
print("\n📊 Bi-LSTM Model Summary:")
model.summary()

trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")

# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict(X_test_lstm).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ BiLSTM (Locked Hyperparameters) + Embeddings for Time-Invariant Vars (race, educ, mother_educ)
# Same locked params: Embedding dim = 4, LSTM units = 128, Dropout = 0.1, Learning rate = 0.001
# Final ~ 0.91
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    LSTM, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector, Flatten, Bidirectional
)
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding (NO id_code embedding)
# ===========================
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)

df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['mammogram'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)

df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0


# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)


# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]

# Only non-time-varying features that are NOT embedded
non_time_varying = ['self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for BiLSTM (1 timestep)
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build FINAL BiLSTM model (LOCKED hyperparameters)
# ===========================
num_input = Input(shape=(1, X_train_lstm.shape[2]), name='num_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# ✅ Locked embedding dim = 4
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine inputs
combined_input = Concatenate()([num_input, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded])

# ✅ BiLSTM instead of single LSTM
x = Bidirectional(LSTM(128))(combined_input)
x = Dropout(0.1)(x)

# Output layer
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, race_input, educ_input, meduc_input], outputs=output)

# ✅ Locked learning rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train FINAL BiLSTM model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_lstm, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_lstm, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)


# ===========================
# 📊 Print Model Summary & Count Params
# ===========================
print("\n📊 BiLSTM + Static Embeddings Model Summary:")
model.summary()

# Count total trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")



# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_lstm, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ BiLSTM (Locked Params) + Super-Regularized ID Embedding
# Goal: ID embedding is weak (can’t memorize individuals), AUC closer to ~0.87–0.89
# Final: 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    LSTM, Bidirectional, Dense, Dropout, Embedding, Input,
    Concatenate, RepeatVector, Flatten, GaussianNoise
)
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding
# ===========================
df['id_code_int'] = df['id_code'].astype('category').cat.codes
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_ids = df['id_code_int'].nunique()
num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)
df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['mammogram'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]
non_time_varying = ['self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_id = train_df['id_code_int'].values
X_test_id = test_df['id_code_int'].values
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for BiLSTM
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build BiLSTM model (Locked Params + Weak ID Embedding)
# ===========================
num_input = Input(shape=(1, X_train_lstm.shape[2]), name='num_input')
id_input = Input(shape=(1,), name='id_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# 🔒 ID embedding (super regularized)
id_embed = Embedding(
    input_dim=num_ids,
    output_dim=1,                            # scalar embedding
    embeddings_regularizer=tf.keras.regularizers.l2(1e-2)
)(id_input)
id_embed = GaussianNoise(0.3)(id_embed)      # strong noise
id_embed = Dropout(0.7)(id_embed)            # very heavy dropout
id_embed_flat = Flatten()(id_embed)
id_embed_expanded = RepeatVector(1)(id_embed_flat)

# 🔒 Time-invariant embeddings
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand for timestep dimension
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine numeric + embeddings
combined_input = Concatenate()([
    num_input, id_embed_expanded, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded
])

# 🔄 Replace LSTM with BiLSTM (still 128 units)
x = Bidirectional(LSTM(128))(combined_input)
x = Dropout(0.1)(x)

# Output
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, id_input, race_input, educ_input, meduc_input], outputs=output)

# 🔒 Locked learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train BiLSTM Model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_lstm, X_train_id, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_lstm, X_test_id, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)


# ===========================
# 📊 Print Model Summary & Count Parameters
# ===========================
print("\n📊 BiLSTM + Super-Regularized ID Embedding Model Summary:")
model.summary()

# Count trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_lstm, X_test_id, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# ROC-AUC
auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU for Mammogram Forecasting (No SMOTE, Locked Hyperparameters, Class Weights)
# 🚫 SMOTE removed — GRU version
# No Embeddings
# AUC ~ 0.75 consistently

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)
df['rolling_mean_lag1_3'] = df.groupby('id_code')['mammogram'].shift(1).rolling(3).mean().reset_index(level=0, drop=True).fillna(0)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = df.groupby('id_code')['self_assement50'].expanding().mean().reset_index(level=0, drop=True)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)

# ===========================
# 🧮 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]
non_time_varying = ['race', 'educ', 'mother_educ', 'self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 📅 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
y_train = train_df[target].astype(int)
X_test = test_df[features]
y_test = test_df[target].astype(int)

# ===========================
# ⚖️ Class weights instead of SMOTE
# ===========================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print(f"\n⚖️ Class Weights: {class_weight_dict}")

# ===========================
# 🔍 Scale
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🔁 Reshape for GRU (same shape as LSTM)
X_train_gru = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_gru = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 GRU Model (Locked Hyperparameters)
# ===========================
model = Sequential([
    GRU(64, input_shape=(X_train_gru.shape[1], X_train_gru.shape[2])),   # ✅ GRU instead of Bi-LSTM
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🛑 Early stopping
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ===========================
# 🚀 Train model (No SMOTE, With Class Weights)
# ===========================
model.fit(
    X_train_gru, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test_gru, y_test),
    callbacks=[early_stop],
    class_weight=class_weight_dict,
    verbose=1
)


# ===========================
# 📊 Print Model Summary & Count Parameters
# ===========================
print("\n📊 GRU Model Summary:")
model.summary()

# Count trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict(X_test_gru).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU (Locked Hyperparameters) + Embeddings for Time-Invariant Vars (race, educ, mother_educ)
# Same settings as your LSTM version:
# - Embedding dim: 4
# - GRU units: 128 (same as LSTM units)
# - Dropout: 0.1
# - Learning rate: 0.001

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding (NO id_code embedding)
# ===========================
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)

df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['mammogram'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)

df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]

# Only non-time-varying features that are NOT embedded
non_time_varying = ['self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for GRU (1 timestep)
X_train_gru = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_gru = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build FINAL GRU model (LOCKED hyperparameters)
# ===========================
num_input = Input(shape=(1, X_train_gru.shape[2]), name='num_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# ✅ Locked embedding dim = 4
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine inputs
combined_input = Concatenate()([num_input, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded])

# ✅ GRU instead of LSTM
x = GRU(128)(combined_input)
x = Dropout(0.1)(x)

# Output
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, race_input, educ_input, meduc_input], outputs=output)

# ✅ Locked learning rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train FINAL model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_gru, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_gru, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)


# ===========================
# 📊 Print Model Summary & Count Parameters
# ===========================
print("\n📊 GRU + Static Embeddings Model Summary:")
model.summary()

# Count total trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")



# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_gru, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU (Locked Params) + Super-Regularized ID Embedding
# Goal: ID embedding too weak to memorize, AUC should drop closer to 0.87–0.89

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    GRU, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector,
    Flatten, GaussianNoise
)
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding
# ===========================
df['id_code_int'] = df['id_code'].astype('category').cat.codes
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_ids = df['id_code_int'].nunique()
num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)
df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['mammogram'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]
non_time_varying = ['self_assement50', 'self_assement60']  # race/educ/mother_educ embedded separately

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_id = train_df['id_code_int'].values
X_test_id = test_df['id_code_int'].values
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for GRU
X_train_gru = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_gru = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# ===========================
# 🏗 Build GRU model (Locked Params + Weak ID Embedding)
# ===========================
num_input = Input(shape=(1, X_train_gru.shape[2]), name='num_input')
id_input = Input(shape=(1,), name='id_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# 🔒 ID embedding (super regularized)
id_embed = Embedding(
    input_dim=num_ids,
    output_dim=1,                             # just scalar
    embeddings_regularizer=tf.keras.regularizers.l2(1e-2)
)(id_input)
id_embed = GaussianNoise(0.3)(id_embed)       # add strong noise
id_embed = Dropout(0.7)(id_embed)             # very heavy dropout
id_embed_flat = Flatten()(id_embed)
id_embed_expanded = RepeatVector(1)(id_embed_flat)

# 🔒 Time-invariant embeddings (locked)
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand for time-step dimension
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# Combine numeric + embeddings
combined_input = Concatenate()([
    num_input, id_embed_expanded, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded
])

# 🔒 GRU (instead of LSTM)
x = GRU(128)(combined_input)
x = Dropout(0.1)(x)

# Output layer
output = Dense(1, activation='sigmoid')(x)

# Model
model = Model(inputs=[num_input, id_input, race_input, educ_input, meduc_input], outputs=output)

# 🔒 Locked learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train Model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_gru, X_train_id, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_gru, X_test_id, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)

# ===========================
# 📊 Print Model Summary & Count Parameters
# ===========================
print("\n📊 GRU + Super-Regularized ID Embedding Model Summary:")
model.summary()

# Count trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_gru, X_test_id, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# ROC-AUC
auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU-D for mammogram Forecasting (No SMOTE, Locked Hyperparameters, Class Weights)
# 🚫 SMOTE removed — GRU-D version
# No Embeddings
# Handles Missingness Explicitly
# auc ~ 0.75

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Dense, Dropout, Input, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)
df['rolling_mean_lag1_3'] = df.groupby('id_code')['mammogram'].shift(1).rolling(3).mean().reset_index(level=0, drop=True).fillna(0)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = df.groupby('id_code')['self_assement50'].expanding().mean().reset_index(level=0, drop=True)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)

# ===========================
# 🧮 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]
non_time_varying = ['race', 'educ', 'mother_educ', 'self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 📅 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=[target])  # ✅ Only drop rows missing the target
test_df = test_df.dropna(subset=[target])
print(f"After dropping NAs -> Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y and ensure float conversion
# ===========================
X_train = train_df[features].astype(float).values
y_train = train_df[target].astype(int)
X_test = test_df[features].astype(float).values
y_test = test_df[target].astype(int)

# ⚖️ Class weights instead of SMOTE
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print(f"\n⚖️ Class Weights: {class_weight_dict}")

# ===========================
# 🔍 Handle Missingness for GRU-D
# ===========================
# 1️⃣ Create masks (1 if present, 0 if missing)
mask_train = np.where(np.isnan(X_train), 0, 1).astype(float)
mask_test = np.where(np.isnan(X_test), 0, 1).astype(float)

# 2️⃣ Fill missing with zero for GRU-D (model learns decay)
X_train_filled = np.nan_to_num(X_train, nan=0)
X_test_filled = np.nan_to_num(X_test, nan=0)

# ===========================
# 🔍 Scale numeric inputs (after filling)
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filled)
X_test_scaled = scaler.transform(X_test_filled)

# 🔁 Reshape for GRU (sequence length = 1 timestep)
X_train_seq = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_seq = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

mask_train_seq = mask_train.reshape((mask_train.shape[0], 1, mask_train.shape[1]))
mask_test_seq = mask_test.reshape((mask_test.shape[0], 1, mask_test.shape[1]))

# ===========================
# 🏗 GRU-D Model (Simplified version of GRU-D)
# ===========================
num_input = Input(shape=(1, X_train_seq.shape[2]), name='num_input')
mask_input = Input(shape=(1, X_train_seq.shape[2]), name='mask_input')

# ✅ GRU takes both numeric values and mask concatenated
combined_input = Concatenate(axis=-1)([num_input, mask_input])

x = GRU(64)(combined_input)   # 🔒 Locked units
x = Dropout(0.3)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[num_input, mask_input], outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🛑 Early stopping
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ===========================
# 🚀 Train model (No SMOTE, With Class Weights)
# ===========================
model.fit(
    [X_train_seq, mask_train_seq], y_train,
    epochs=50,
    batch_size=32,
    validation_data=([X_test_seq, mask_test_seq], y_test),
    callbacks=[early_stop],
    class_weight=class_weight_dict,
    verbose=1
)

# ===========================
# 📊 Print Model Summary & Count Trainable Parameters
# ===========================
print("\n📊 GRU-D Model Summary:")
model.summary()

# Count trainable parameters
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")


# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_seq, mask_test_seq]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

print("\n🧾 Confusion Matrix:")
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU-D (Locked Hyperparameters) + Embeddings for Time-Invariant Vars (race, educ, mother_educ)
# Handles missingness with masks and decay terms (GRU-D style)
# Embedding dim: 4 | GRU units: 128 | Dropout: 0.1 | LR: 0.001

# AUC ~ 0.85

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector, Flatten, Multiply
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding (NO id_code embedding)
# ===========================
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)
df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['mammogram'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]

non_time_varying = ['self_assement50', 'self_assement60']

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016
print(f"\n📅 Forecasting for {test_year} (target = 2018)")

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

print(f"Before dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")
train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])
print(f"After dropping NAs → Train: {train_df.shape}, Test: {test_df.shape}")

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 🔍 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ===========================
# 🧩 GRU-D style missingness handling
# ===========================
# ✅ 1. Create masks (1 if present, 0 if missing) → convert to NumPy!
mask_train = (~X_train.isna()).astype(float).values
mask_test = (~X_test.isna()).astype(float).values

# ✅ 2. Replace NaNs with 0 (GRU-D learns to use mask for missing info)
X_train_filled = np.nan_to_num(X_train_scaled, nan=0)
X_test_filled = np.nan_to_num(X_test_scaled, nan=0)

# ✅ 3. Reshape for GRU input
X_train_gru = X_train_filled.reshape((X_train_filled.shape[0], 1, X_train_filled.shape[1]))
X_test_gru = X_test_filled.reshape((X_test_filled.shape[0], 1, X_test_filled.shape[1]))

mask_train_gru = mask_train.reshape((mask_train.shape[0], 1, mask_train.shape[1]))
mask_test_gru = mask_test.reshape((mask_test.shape[0], 1, mask_test.shape[1]))

# ===========================
# 🏗 GRU-D Model (Locked Hyperparameters)
# ===========================
num_input = Input(shape=(1, X_train_gru.shape[2]), name='num_input')
mask_input = Input(shape=(1, X_train_gru.shape[2]), name='mask_input')

# ✅ Apply mask to input (GRU-D style)
masked_input = Multiply()([num_input, mask_input])

race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# ✅ Locked embedding dim = 4
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand for time-step dimension
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# ✅ Combine everything
combined_input = Concatenate()([masked_input, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded])

# ✅ GRU layer (128 units, dropout = 0.1)
x = GRU(128)(combined_input)
x = Dropout(0.1)(x)

# Output layer
output = Dense(1, activation='sigmoid')(x)

# ✅ Build GRU-D model
model = Model(inputs=[num_input, mask_input, race_input, educ_input, meduc_input], outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train GRU-D Model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_gru, mask_train_gru, X_train_race, X_train_educ, X_train_meduc], y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_gru, mask_test_gru, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)


# ===========================
# 📊 Print Model Summary & Count Trainable Parameters
# ===========================
print("\n📊 GRU-D + Static Embeddings Model Summary:")
model.summary()

# Count trainable parameters for reporting in your table
trainable_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {trainable_params}")



# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_gru, mask_test_gru, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ GRU-D (Locked Params) + Super-Regularized ID Embedding
# Goal: GRU-D handles missingness with masks; ID embedding heavily regularized
# Expected: ID embedding too weak to memorize, AUC should drop closer to 0.87–0.89

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    GRU, Dense, Dropout, Embedding, Input, Concatenate, RepeatVector,
    Flatten, GaussianNoise, Multiply
)
from tensorflow.keras.callbacks import EarlyStopping

# ===========================
# 📂 Load & Sort Panel Data
# ===========================
df = pd.read_csv("XGBoost.csv")
df = df.sort_values(by=["id_code", "year"])

# ===========================
# 🔢 Prepare categorical vars for embedding
# ===========================
df['id_code_int'] = df['id_code'].astype('category').cat.codes
df['race_int'] = df['race'].astype('category').cat.codes
df['educ_int'] = df['educ'].astype('category').cat.codes
df['mother_educ_int'] = df['mother_educ'].astype('category').cat.codes

num_ids = df['id_code_int'].nunique()
num_race = df['race_int'].nunique()
num_educ = df['educ_int'].nunique()
num_meduc = df['mother_educ_int'].nunique()

# ===========================
# 🎯 Forecast target + engineered features
# ===========================
df['mammogram_t_plus_1'] = df.groupby('id_code')['mammogram'].shift(-1)
df['mammogram_lag1'] = df.groupby('id_code')['mammogram'].shift(1)
df['mammogram_lag2'] = df.groupby('id_code')['mammogram'].shift(2)
df['rolling_mean_lag1_3'] = (
    df.groupby('id_code')['mammogram'].shift(1)
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
      .fillna(0)
)
df['trend_income_log'] = df.groupby('id_code')['income_log'].diff().fillna(0)
df['cumulative_avg_self_assess50'] = (
    df.groupby('id_code')['self_assement50']
      .expanding().mean()
      .reset_index(level=0, drop=True)
)
df['first_year'] = df.groupby('id_code').cumcount() == 0

# interraction:
df['income_log_mammogram_lag1'] = df.groupby('id_code')['income_log'].shift(1) * df.groupby('id_code')['mammogram'].shift(1)

# ===========================
# 📋 Features
# ===========================
time_varying = [
    'marital_status', 'time_step', 'health_plan', 'region', 'year',
    'hh_children', 'health_facility', 'health_provider',
    'income_log_self50', 'income_log_self60',
    'income_log', 'region_year', 'region_health_plan',
    'race_health_plan', 'income_log_region', 'income_log_health_provider',
    'income_log_mammogram_lag1', 'educ_mother_educ', 'trend_income_log',
    'rolling_mean_lag1_3', 'mammogram_lag1'
]
non_time_varying = ['self_assement50', 'self_assement60']  # race/educ/mother_educ embedded separately

features = time_varying + non_time_varying
target = 'mammogram_t_plus_1'

# ===========================
# 🗓 Forecasting for 2016 → predict 2018
# ===========================
train_years = [2008, 2010, 2012, 2014, 2016]
test_year = 2016

train_df = df[df['year'].isin(train_years)]
test_df = df[df['year'] == test_year]

train_df = train_df.dropna(subset=features + [target])
test_df = test_df.dropna(subset=features + [target])

# ===========================
# 🔄 Split X/y
# ===========================
X_train = train_df[features]
X_test = test_df[features]
y_train = train_df[target].astype(int)
y_test = test_df[target].astype(int)

# Extract categorical vars for embedding
X_train_id = train_df['id_code_int'].values
X_test_id = test_df['id_code_int'].values
X_train_race = train_df['race_int'].values
X_test_race = test_df['race_int'].values
X_train_educ = train_df['educ_int'].values
X_test_educ = test_df['educ_int'].values
X_train_meduc = train_df['mother_educ_int'].values
X_test_meduc = test_df['mother_educ_int'].values

# ===========================
# 📏 Scale numeric features
# ===========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ===========================
# 🧩 GRU-D style missingness handling
# ===========================
mask_train = (~X_train.isna()).astype(float).values
mask_test = (~X_test.isna()).astype(float).values

X_train_filled = np.nan_to_num(X_train_scaled, nan=0)
X_test_filled = np.nan_to_num(X_test_scaled, nan=0)

# Reshape for GRU (and mask)
X_train_gru = X_train_filled.reshape((X_train_filled.shape[0], 1, X_train_filled.shape[1]))
X_test_gru = X_test_filled.reshape((X_test_filled.shape[0], 1, X_test_filled.shape[1]))

mask_train_gru = mask_train.reshape((mask_train.shape[0], 1, mask_train.shape[1]))
mask_test_gru = mask_test.reshape((mask_test.shape[0], 1, mask_test.shape[1]))

# ===========================
# 🏗 Build GRU-D model (Locked Params + Weak ID Embedding)
# ===========================
num_input = Input(shape=(1, X_train_gru.shape[2]), name='num_input')
mask_input = Input(shape=(1, X_train_gru.shape[2]), name='mask_input')

# ✅ GRU-D style: apply mask to input
masked_input = Multiply()([num_input, mask_input])

# ID + other embeddings
id_input = Input(shape=(1,), name='id_input')
race_input = Input(shape=(1,), name='race_input')
educ_input = Input(shape=(1,), name='educ_input')
meduc_input = Input(shape=(1,), name='meduc_input')

# 🔒 ID embedding (super regularized)
id_embed = Embedding(input_dim=num_ids, output_dim=1,
                     embeddings_regularizer=tf.keras.regularizers.l2(1e-2))(id_input)
id_embed = GaussianNoise(0.3)(id_embed)
id_embed = Dropout(0.7)(id_embed)
id_embed_flat = Flatten()(id_embed)
id_embed_expanded = RepeatVector(1)(id_embed_flat)

# 🔒 Time-invariant embeddings
race_embed = Embedding(input_dim=num_race, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(race_input)
educ_embed = Embedding(input_dim=num_educ, output_dim=4,
                       embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(educ_input)
meduc_embed = Embedding(input_dim=num_meduc, output_dim=4,
                        embeddings_regularizer=tf.keras.regularizers.l2(1e-4))(meduc_input)

# Flatten & expand embeddings
race_embed_expanded = RepeatVector(1)(Flatten()(race_embed))
educ_embed_expanded = RepeatVector(1)(Flatten()(educ_embed))
meduc_embed_expanded = RepeatVector(1)(Flatten()(meduc_embed))

# ✅ Combine numeric, mask-handled input, and embeddings
combined_input = Concatenate()([
    masked_input, id_embed_expanded, race_embed_expanded, educ_embed_expanded, meduc_embed_expanded
])

# ✅ GRU (locked params)
x = GRU(128)(combined_input)
x = Dropout(0.1)(x)

# Output layer
output = Dense(1, activation='sigmoid')(x)

# ✅ Build GRU-D model
model = Model(inputs=[num_input, mask_input, id_input, race_input, educ_input, meduc_input],
              outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ===========================
# 🚀 Train Model
# ===========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit([X_train_gru, mask_train_gru, X_train_id, X_train_race, X_train_educ, X_train_meduc],
          y_train,
          epochs=60,
          batch_size=32,
          validation_data=([X_test_gru, mask_test_gru, X_test_id, X_test_race, X_test_educ, X_test_meduc], y_test),
          callbacks=[early_stop],
          verbose=1)

# ✅ Print model summary and count parameters
model.summary()

total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
print(f"\n🔢 Total Trainable Parameters: {total_params}")


# ===========================
# 📈 Evaluate
# ===========================
y_pred_probs = model.predict([X_test_gru, mask_test_gru, X_test_id, X_test_race, X_test_educ, X_test_meduc]).ravel()
y_pred = (y_pred_probs > 0.5).astype(int)

print("\n✅ Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

# Confusion Matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix ({test_year} → 2018)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# ROC-AUC
auc_score = roc_auc_score(y_test, y_pred_probs)
print(f"\n🔵 ROC AUC Score: {auc_score:.4f}")
fpr, tpr, _ = roc_curve(y_test, y_pred_probs)
plt.plot(fpr, tpr, label=f'AUC = {auc_score:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ROC Curve (2016 → 2018)")
plt.legend()
plt.grid(True)
plt.show()
